In [1]:
import time
import json
import pika
import uuid
import sys
import pybars
import requests
import psycopg2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#-------------------Conexión con la base de datos de Chatblocks -------------------
#DNS="host='localhost' dbname='ChatBlocksVF' user='postgres' password='2011080606'"
DNS="host='localhost' dbname='chatblocks' user='postgres' password='123456789'"
con=psycopg2.connect(DNS)
cur=con.cursor()

In [3]:
try:
    DNS="host='localhost' dbname='chatblocks' user='postgres' password='123456789'"
    con=psycopg2.connect(DNS)
    cur=con.cursor()
    print("conexion exitosa")
except (Exception, psycopg2.Error) as error:
    print("Error en conexion", error) 

conexion exitosa


In [4]:
def consulta(query):
    try:
        DNS="host='localhost' dbname='chatblocks' user='postgres' password='123456789'"
        con=psycopg2.connect(DNS)
        cur=con.cursor()
        cur.execute(query)
        dato=cur.fetchone()
    except (Exception, psycopg2.Error) as error:
        print("Error en consulta", error) 
    return dato

In [5]:
def consulta2(query):
    global cur
    cur.execute(query)
    dato=cur.fetchall()
    return dato

In [6]:
def insertar(query):
    global cur
    global con
    cur.execute(query)
    con.commit()

In [7]:
def cerrarConexionDB():
    global cur
    global con
    cur.close()
    con.close()

In [8]:
#-----PASO 9. Variante 1: Creación de la respuesta que pertenece a un bloque simple: Informativo-----
def mensInf(state,mensaje):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":"" }}
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
    
    #-------PASO 9. Variante 1: Consultas a la base de datos para la creación de la respuesta--------        
    decoded["Respuesta"]["content"] = str(consulta("select contenido from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["next_id"] = str(consulta("select next_id from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["blockType"]= str(consulta("select blockType from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["contentType"]= str(consulta("select contentType from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["typingTime"]= str(consulta("select typingTime from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    
    mensajeAct=str(mensaje["payload"]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='informativo' where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)
    decoded["Respuesta"]["payload"]= json.loads(pay)
    
    #decoded["Respuesta"]["payload"].append(json.loads(pay))
    #----------------------PASO 10. Inicio: Retorno de la respuesta generada----------------------
    return decoded

In [9]:

def mensInfDatosU(mensaje,content):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":" "}}
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
    
    decoded["Respuesta"]["content"] = str(content)
    decoded["Respuesta"]["blockType"]= 'informativo'
    decoded["Respuesta"]["contentType"]= 'text'
    decoded["Respuesta"]["typingTime"]= 1
    
    payload={"content":"","recipient_id":mensaje["payload"]["recipient_id"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)
    decoded["Respuesta"]["payload"]=json.loads(pay)
    #decoded["Respuesta"]["payload"]decoded["Respuesta"]["payload"].append(json.loads(pay))
    return decoded

In [10]:
#-----PASO 9. Variante 1: Creación de la respuesta que pertenece a un bloque simple: QuickReply-----
def mensQr(state,mensaje):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":" "}}
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
    
    #-------PASO 9. Variante 1: Consultas a la base de datos para la creación de la respuesta--------
    opc=(consulta2("select opciones from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    opciones=opc[0].split(",")
    
    decoded["Respuesta"]["content"] = str(consulta("select contenido from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["options"] = opciones
    decoded["Respuesta"]["next_id"] = str(consulta("select next_id from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["blockType"]= str(consulta("select blockType from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"]["typingTime"]= str(consulta("select typingTime from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["default_id"] = str(consulta("select default_id from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    #decoded["Respuesta"][0]["save_var"] = str(consulta("select save_var from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["save_var"] = str(consulta("select var from bloquequickreply join variables on variables.id_var =  cast ( bloquequickreply.id_var as INTEGER) where bloquequickreply.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloquequickreply.namestate='"+state+"'")[0])
    
    mensajeAct=str(mensaje["payload"]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='quickReply' where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)
    decoded["Respuesta"]["payload"]=json.loads(pay)
    #decoded["Respuesta"]["payload"].append(json.loads(pay))
    #----------------------PASO 10. Inicio: Retorno de la respuesta generada----------------------
    return decoded

In [11]:
#-----PASO 9. Variante 1: Creación de la respuesta que pertenece a un bloque simple: Input-----
def mensInp(state,mensaje):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":" "}}
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
    
    #-------PASO 9. Variante 1: Consultas a la base de datos para la creación de la respuesta--------
    decoded["Respuesta"]["content"] = str(consulta("select contenido from bloqueinput where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"]["next_id"] = str(consulta("select next_id from bloqueinput where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["blockType"]= str(consulta("select blockType from bloqueinput where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"]["contentType"]= str(consulta("select contentType from bloqueinput where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"]["typingTime"]= str(consulta("select typingTime from bloqueinput where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["validacion"]= str(consulta("select validacion from bloqueinput where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["Default_id"] = str(consulta("select default_id from bloqueinput where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    #decoded["Respuesta"][0]["save_var"] = str(consulta("select save_var from bloqueinput where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["save_var"] = str(consulta("select var from bloqueinput join variables on variables.id_var =  cast ( bloqueinput.id_var as INTEGER) where bloqueinput.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueinput.namestate='"+state+"'")[0])
    mensajeAct=str(mensaje["payload"]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='input' where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)
    decoded["Respuesta"]["payload"] = json.loads(pay)
    #decoded["Respuesta"]["payload"].append(json.loads(pay))
    #----------------------PASO 10. Inicio: Retorno de la respuesta generada----------------------
    return decoded

In [12]:
#-----PASO 9. Variante 1: Creación de la respuesta que pertenece a un bloque simple: Slide-----
def mensSlide(state,mensaje):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":" "}}
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
    
    #-------PASO 9. Variante 1: Consultas a la base de datos para la creación de la respuesta--------
    id_block,next_id,blocktype,typingtime,default_id = consulta("select id_block,next_id,blocktype,typingtime,default_id from bloqueslide where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")
    save_var = str(consulta("select var from bloqueslide join variables on variables.id_var =  cast ( bloqueslide.id_var as INTEGER) where bloqueslide.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueslide.namestate='"+state+"'")[0]) 
    data_elements=consulta2("select * from elementos where id_block='"+str(id_block)+"' and blocktype='slide'")    
    
    elements=[]
    
    for element in data_elements:
        for elm in element:
            print('Elemento-> ',elm)        
        buttons =[]    
        data_buttons=consulta2("select * from botones where id_elemento='"+str(element[5])+"'")
        for btn in data_buttons:
            if btn[2]=='postback':
                button={'titleButton':btn[1],'typeButton':'postback','payloadButton':btn[3]}
            if btn[2]=='web_url':
                button={'titleButton':btn[1],'typeButton':'web_url','urlButton':btn[3]}
            buttons.append(button)                
        data={'title':element[1], 'image_url':element[2], 'subtitle':element[3], 'buttons':buttons}
        elements.append(data)    
    opciones=next_id.split(",")        
    decoded["Respuesta"]["content"] = elements
    decoded["Respuesta"]["next_id"] = opciones
    decoded["Respuesta"]["blockType"]= str(blocktype)    
    decoded["Respuesta"]["typingTime"]= str(typingtime)
    decoded["Respuesta"]["default_id"] = str(default_id)
    decoded["Respuesta"]["save_var"] = str(save_var)
    
    mensajeAct=str(mensaje["payload"]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='slide' where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)
    decoded["Respuesta"]["payload"]=json.loads(pay)
    #decoded["Respuesta"]["payload"].append(json.loads(pay))
    #----------------------PASO 10. Inicio: Retorno de la respuesta generada----------------------
    return decoded

In [13]:
#-----PASO 9. Variante 1: Creación de la respuesta que pertenece a un bloque simple: SlideBuy-----
def mensSlideBuy(state,mensaje):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":" "}}
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
    
    #-------PASO 9. Variante 1: Consultas a la base de datos para la creación de la respuesta--------
    id_block,next_id,blocktype,typingtime,default_id=consulta("select id_block,next_id,blocktype,typingtime,default_id from bloqueslidebuy where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")
    save_var = str(consulta("select var from bloqueslidebuy join variables on variables.id_var =  cast ( bloqueslidebuy.id_var as INTEGER) where bloqueslidebuy.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueslidebuy.namestate='"+state+"'")[0]) 
    data_elements=consulta2("select * from elementosbuy where id_block='"+str(id_block)+"'")
    elements=[]
    
    for element in data_elements:
        buttons =[]
        button={'titleButton':'Comprar','typeButton':'postback','payloadButton':element[0]}
        buttons.append(button)       
        subT=element[6]+'.  Precio: $'+element[7]
        data={'title':element[4], 'image_url':element[5], 'subtitle':subT, 'buttons':buttons}
        elements.append(data)
    
    decoded["Respuesta"]["content"] = elements
    decoded["Respuesta"]["next_id"] = str(next_id)
    decoded["Respuesta"]["blockType"]= str(blocktype)    
    decoded["Respuesta"]["typingTime"]= str(typingtime)
    decoded["Respuesta"]["default_id"] = str(default_id)
    decoded["Respuesta"]["save_var"] = str(save_var)
    
    mensajeAct=str(mensaje["payload"]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='slideBuy' where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)
    decoded["Respuesta"]["payload"] =json.loads(pay)
    #decoded["Respuesta"]["payload"].append(json.loads(pay))
    #----------------------PASO 10. Inicio: Retorno de la respuesta generada----------------------
    return decoded

In [14]:
#-----PASO 9. Variante 1: Creación de la respuesta que pertenece a un bloque simple: Ticket-----
def mensTicket(state,mensaje):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":" "}}
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
    
    #-------PASO 9. Variante 1: Consultas a la base de datos para la creación de la respuesta--------
    dataTicket=consulta2("select * from bloqueticket where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0]
    rescue_var=dataTicket[7].split(",")
    id_blockT=consulta("select id_block from bloqueslidebuy where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+rescue_var[1]+"'")[0]
    
    elements=[]
    subtotal=0
    for obj in mensaje["payload"]["entities"][rescue_var[0]]:
        data_elements=consulta2("select * from elementosbuy where id_busqueda='"+obj+"' and id_block='"+id_blockT+"'")[0]
        subtotal=subtotal+float(data_elements[4])*int(mensaje["payload"]["entities"][rescue_var[0]][obj])
        data={'title':data_elements[4],'subtitle':data_elements[6],'quantity':mensaje["payload"]["entities"][rescue_var[0]][obj],'price':data_elements[7],'currency':dataTicket[6],'image_url':data_elements[5]}
        elements.append(data)
        
    total=subtotal+float(dataTicket[8])+float(dataTicket[9])        
    summary={"subtotal":subtotal,"shipping_cost":float(dataTicket[8]),"total_tax":float(dataTicket[9]),"total_cost":total}
    address={"street_1":mensaje["payload"]["entities"][dataTicket[10]],"street_2":mensaje["payload"]["entities"][dataTicket[11]],"city":mensaje["payload"]["entities"][dataTicket[12]],"postal_code":mensaje["payload"]["entities"][dataTicket[13]],"state":mensaje["payload"]["entities"][dataTicket[14]],"country":mensaje["payload"]["entities"][dataTicket[15]]}
    content={"template_type":"receipt","recipient_name":mensaje["payload"]["entities"][dataTicket[17]],"order_number":"12345678902","currency":dataTicket[6],"payment_method":mensaje["payload"]["entities"][dataTicket[16]],"address":address,"summary":summary,"elements":elements}
    
    decoded["Respuesta"]["content"] = content
    decoded["Respuesta"]["next_id"] = str(dataTicket[3])
    decoded["Respuesta"]["blockType"]= str(dataTicket[4])    
    decoded["Respuesta"]["typingTime"]= str(dataTicket[5])
    
    mensaje["payload"]["entities"]={}
    mensajeAct=str(mensaje["payload"]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='ticket' where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)
    decoded["Respuesta"]["payload"]=json.loads(pay)
    #----------------------PASO 10. Inicio: Retorno de la respuesta generada----------------------
    return decoded

In [15]:
#-----PASO 9. Variante 2: Creación de la respuesta que pertenece a un bloque dinámico: Informativo-----
def mensInfDin(state,mensaje):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":" "}}
    
    id_bloque=consulta("select id_block from bloqueinformativoDinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0]
    links=consulta2("select links from linksapis where id_block='"+str(id_bloque)+"' and blocktype='informativoDinamico'")
    name=consulta2("select namecredencial from credencialesapis where id_block='"+str(id_bloque)+"' and blocktype='informativoDinamico'")
    cr=consulta2("select credencial from credencialesapis where id_block='"+str(id_bloque)+"' and blocktype='informativoDinamico'")
    contenido=consulta("select contenido from bloqueinformativoDinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0]
    credenciales={}
    cont=0
    for nm in name:
        credenciales[nm[0]]=cr[cont][0]        
        cont=cont+1
    #----------------PASO 9. Variante 2: Consulta a una API para obtener su información----------------
    respAPI=consultarAPIs(links, credenciales, mensaje["payload"]["entities"])
    output=ingresarVar(contenido, "",mensaje["payload"]["entities"])
    #---PASO 9. Variante 2: Aplica handlebars para el procesamiento y la creacion del contenido de la respuesta---
    content=crearRespuesta(output, respAPI, mensaje["payload"]["entities"])
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
        
    decoded["Respuesta"]["content"] = content   
    decoded["Respuesta"]["next_id"] = str(consulta("select next_id from bloqueinformativoDinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["blockType"]= str(consulta("select blockType from bloqueinformativoDinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"]["contentType"]= str(consulta("select contentType from bloqueinformativoDinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"]["typingTime"]= str(consulta("select typingTime from bloqueinformativoDinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    
    mensajeAct=str(mensaje["payload"]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='informativoDinamico' where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)    
    decoded["Respuesta"]["payload"]=json.loads(pay)
    #-----------------------PASO 10. Inicio: Retorno de la respuesta generada-----------------------
    return decoded

In [16]:
#-----PASO 9. Variante 2: Creación de la respuesta que pertenece a un bloque dinámico: Input-----
def mensInpDin(state,mensaje):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":" "}}
    
    id_bloque=consulta("select id_block from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0]    
    #save_var=consulta("select save_var from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0]
    save_var=consulta("select var from bloqueinputdinamico join variables on variables.id_var =  cast ( bloqueinputdinamico.id_var as INTEGER) where bloqueinputdinamico.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueinputdinamico.namestate='"+state+"'")[0]
    links=consulta2("select links from linksapis where id_block='"+str(id_bloque)+"' and blocktype='inputDinamico'")
    name=consulta2("select namecredencial from credencialesapis where id_block='"+str(id_bloque)+"' and blocktype='inputDinamico'")
    cr=consulta2("select credencial from credencialesapis where id_block='"+str(id_bloque)+"' and blocktype='inputDinamico'")
    contenido=consulta("select contenido from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0]
    credenciales={}
    cont=0
    for nm in name:
        credenciales[nm[0]]=cr[cont][0]        
        cont=cont+1
    #----------------PASO 9. Variante 2: Consulta a una API para obtener su información----------------
    respAPI=consultarAPIs(links, credenciales, mensaje["payload"]["entities"])
    output=ingresarVar(contenido, save_var, mensaje["payload"]["entities"])
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
        
    decoded["Respuesta"]["content"] = output   
    decoded["Respuesta"]["next_id"] = str(consulta("select next_id from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["blockType"]= str(consulta("select blockType from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"]["contentType"]= str(consulta("select contentType from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"]["typingTime"]= str(consulta("select typingTime from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["validacion"]= str(consulta("select validacion from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["Default_id"] = str(consulta("select default_id from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    #decoded["Respuesta"][0]["save_var"] = str(consulta("select save_var from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["save_var"] = str(consulta("select var from bloqueinputdinamico join variables on variables.id_var =  cast ( bloqueinputdinamico.id_var as INTEGER) where bloqueinputdinamico.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueinputdinamico.namestate='"+state+"'")[0])
    mensajeAct=str(mensaje["payload"]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='inputDinamico' where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)
    decoded["Respuesta"]["payload"]=json.loads(pay)
    #-----------------------PASO 10. Inicio: Retorno de la respuesta generada-----------------------
    return decoded

In [17]:
#-----PASO 9. Variante 2: Creación de la respuesta que pertenece a un bloque dinámico: QuickReply-----
def mensQRDin(state,mensaje):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":" "}}
    
    id_bloque=consulta("select id_block from bloqueqrdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0]
    links=consulta2("select links from linksapis where id_block='"+str(id_bloque)+"' and blocktype='quickReplyDinamico'")
    name=consulta2("select namecredencial from credencialesapis where id_block='"+str(id_bloque)+"' and blocktype='quickReplyDinamico'")
    cr=consulta2("select credencial from credencialesapis where id_block='"+str(id_bloque)+"' and blocktype='quickReplyDinamico'")    
    credenciales={}
    cont=0
    for nm in name:
        credenciales[nm[0]]=cr[cont][0]        
        cont=cont+1
        
    print("Datos bloque QR Dinamico=>")
    print("id_bloque-> ",id_bloque)
    print("links-> ",links)
    print("credenciales-> ",credenciales)
    #----------------PASO 9. Variante 2: Consulta a una API para obtener su información----------------
    respAPI=consultarAPIs(links, credenciales, mensaje["payload"]["entities"])
    #---PASO 9. Variante 2: Aplica handlebars para el procesamiento y la creacion del contenido de la respuesta---
    options=generarOpc(respAPI)
    options2=(((str(options).replace("'", "")).replace("[", "")).replace("]","")).replace(" ","")
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
        
    decoded["Respuesta"]["content"] = str(consulta("select contenido from bloqueqrdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["options"] = options
    decoded["Respuesta"]["next_id"] = str(consulta("select next_id from bloqueqrdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["blockType"]= str(consulta("select blockType from bloqueqrdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])        
    decoded["Respuesta"]["typingTime"]= str(consulta("select typingTime from bloqueqrdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"]["Default_id"] = str(consulta("select default_id from bloqueqrdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    #decoded["Respuesta"][0]["save_var"] = str(consulta("select save_var from bloqueqrdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["save_var"] = str(consulta("select var from bloqueqrdinamico join variables on variables.id_var =  cast ( bloqueinput.id_var as INTEGER) where bloqueqrdinamico.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueqrdinamico.namestate='"+state+"'")[0])
    mensajeAct=str(mensaje["payload"]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='quickReplyDinamico' where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
    insertar("update bloqueqrdinamico set opciones='"+options2+"' where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)
    decoded["Respuesta"]["payload"]=json.loads(pay)
    #-----------------------PASO 10. Inicio: Retorno de la respuesta generada-----------------------
    return decoded

In [18]:
#-----PASO 9. Variante 2: Creación de la respuesta que pertenece a un bloque dinámico: Slide-----
def mensSlideDin(state,mensaje):
    n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":" "}}
    
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
    
    id_bloque=consulta("select id_block from bloqueslidedinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0]
    links=consulta2("select links from linksapis where id_block='"+str(id_bloque)+"' and blocktype='slideDinamico'")
    
    nameC=consulta2("select namecredencial from credencialesapis where id_block='"+str(id_bloque)+"' and blocktype='slideDinamico'")
    cr=consulta2("select credencial from credencialesapis where id_block='"+str(id_bloque)+"' and blocktype='slideDinamico'")
    contenido=consulta("select contenido from bloqueslidedinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0]
    #contenido='[{{#each stations}} {"title": "Estacion con id {{id}}", "image_url": "https://www.ecobici.cdmx.gob.mx/sites/default/files/u14/en_bicicletas-2.png", "subtitle": "Direccion = {{address}}" }, {{/each}}{"Fin":" "}]'
    credenciales={}
    cont=0
    for nm in nameC:
        credenciales[nm[0]]=cr[cont][0]     
        print(credenciales[nm[0]])
        cont=cont+1
    #----------------PASO 9. Variante 2: Consulta a una API para obtener su información----------------
    respAPI=consultarAPIs(links, credenciales, mensaje["payload"]["entities"])
    output=ingresarVar(contenido, "",mensaje["payload"]["entities"])
    #---PASO 9. Variante 2: Aplica handlebars para el procesamiento y la creacion del contenido de la respuesta---
    resp=crearRespuesta2(output, respAPI, mensaje["payload"]["entities"])
    options = json.loads(resp)

    content=[]
    if 'actual_pos' in mensaje["payload"]["entities"]:
        cont=mensaje["payload"]["entities"]['actual_pos']
    else:
        cont=0
        
    contAux=0
    while contAux<10:
        if cont<len(options):
            content.append(options[cont])
        else:
            cont=0
            content.append(options[cont])
        contAux=contAux+1
        cont=cont+1
        
    decoded["Respuesta"]["content"] = content
    decoded["Respuesta"]["next_id"] = str(consulta("select next_id from bloqueslidedinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"]["blockType"]= str(consulta("select blockType from bloqueslidedinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])    
    #decoded["Respuesta"][0]["contentType"]= str(consulta("select contentType from bloqueslidedinamico where id_robot='"+mensaje["payload"]["robot"]+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"]["typingTime"]= str(consulta("select typingTime from bloqueslidedinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'")[0])

    mensaje["payload"]["entities"]['actual_pos']=cont
    mensajeAct=str(mensaje["payload"]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='slideDinamico' where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
    pay=json.dumps(payload)
    decoded["Respuesta"]["payload"]=json.loads(pay)
    #-----------------------PASO 10. Inicio: Retorno de la respuesta generada-----------------------
    return decoded

In [19]:
#-------se generan las opciones para un QuickReply-------
def generarOpc(re):
    key=""
    if(len(re)>1):
        print("Mayor a 1")   
    else:
        for raiz in re:
            key=raiz
        opciones=list(re[key][0].keys())
    
    return opciones

In [20]:
#-------Se busca y se ingresa las bariables contenidas en las "entities" en una cadena-------
def ingresarVar(content, nameVar, entities):
    prueba1={}
    prueba2={}
    principal=""    
    
    output, var=completarURLs(content, prueba1, prueba2, entities)
    if(nameVar!=""):
        entities[nameVar]=var        
    
    return output

In [21]:
#-------Se genera el contenido de la respuesta mediante handlebars-------
def crearRespuesta2(content, re, entities):
    nameVar='vacio'
    cont=0
    cont2=0
    
    source = content
    #print('Source-> ',source)
    compiler = pybars.Compiler()
    template = compiler.compile(source)
    output = template(re)
    return output

In [22]:
#-------Consulta a las APIs mediante operaciones de tipo GET-------
def consultarAPIs(links, tockens, entities):
    print("consultarAPIs-> ")
    print("links-> ", links)
    print("tockens-> ", tockens)
    print("entities-> ", entities)
    re={}
    prueba1={}
    prueba2={}
    if(len(links)>0): 
        for link in links:      
            #print("\n\nDatos enviados de consultarAPIs a completarURLs")
            #print("\n\nURl 1= ",links[link])
            #print("\n\nTockens = ",tockens)
            #print("\n\nConsulta 1= ",re)
            #print("\n\nEtidades 1= ",entities)
            urlF, var=completarURLs(link[0], tockens, re, entities)           
            r = requests.get(urlF)
            print(r);
            #print(urlF)
            re=r.json()  
    return re            

In [23]:
#-------Se genera el contenido de la respuesta mediante handlebars-------
def crearRespuesta(content, re, entities):
    nameVar='vacio'
    cont=0
    cont2=0
    key=''
    if(len(re)>1):
        print('crearRespuesta-> if')
        source = content
        print('Source-> ',source)
        compiler = pybars.Compiler()
        template = compiler.compile(source)
        output = template(re)    
    else:
        print('crearRespuesta-> else')
        for raiz in re:
            key=raiz
        general=re[key]
        for var in entities:
            nameVar=var  
            cont=cont+1;
        for var in entities:
            cont2=cont2+1;
            if cont2<cont:
                nameVar2=var
        #print("general= ",general)
        print("nameVar= ",nameVar2)
        print("entities[nameVar]= ",entities[nameVar])
        #posicion=buscarDatos(general, nameVar2, entities[nameVar])
        posicion=buscarDatos(general, entities[nameVar2], entities[nameVar])
        print('template',re[key][posicion])
        source = content
        compiler = pybars.Compiler()
        template = compiler.compile(source)
        output = template(re[key][posicion]) 
    print('crearRespuesta-> ',output)
    return output

In [24]:
#------------Función que completa los links mediante variables o credenciales-----------------------
def completarURLs(url, tockens, consulta, entities):
    pos1=1
    pos2=0
    posAux=0
    cadAux=url
    
    var=" "
    while(pos1>-1):
        if(posAux<len(url)):
            pos1=cadAux.find('{')
            pos2=cadAux.find('}')
            if(url[posAux+pos1+1]=='{' and pos1>-1): 
                cadAux=url[pos2+posAux+2:]
                token=url[posAux+pos1+2:pos2+posAux]
                if token in entities:
                    res=len(entities[token])-len(token)
                    url=url[:posAux+pos1+2]+entities[token]+url[pos2+posAux:]
                    posAux=pos1+posAux+res+2
                else:
                    posAux=pos2+posAux+2
            elif(pos1>-1):                                
                cadAux=url[pos2+posAux+1:]                
                token2=url[pos1+posAux+1:pos2+posAux]
                token=token2
                if token.lower() in tockens:
                    url=url[:pos1+posAux]+tockens[token.lower()]+url[pos2+1+posAux:]
                    cadAux=url[pos1+posAux:]
                    posAux=pos1+posAux
                if token.upper() in tockens:
                    url=url[:pos1+posAux]+tockens[token.upper()]+url[pos2+1+posAux:]
                    cadAux=url[pos1+posAux:]
                    posAux=pos1+posAux
                elif token.lower() in consulta:
                    url=url[:pos1+posAux]+consulta[token.lower()]+url[pos2+1+posAux:]
                    cadAux=url[pos1+posAux:]
                    posAux=pos1+posAux
                elif token.upper() in consulta:
                    url=url[:pos1+posAux]+consulta[token.upper()]+url[pos2+1+posAux:]
                    cadAux=url[pos1+posAux:]
                    posAux=pos1+posAux
                elif token2 in entities:
                    url=url[:pos1+posAux]+entities[token2]+url[pos2+1+posAux:]
                    cadAux=url[pos1+posAux:]
                    posAux=pos1+posAux
                else:
                    posAux=pos2+posAux+1
        else:
            pos1=-1
    print(url)
    return url, var

In [25]:
#-------Busqueda de datos en el JSON de la respuesta de las APIs-------
def buscarDatos(general, nameVar, dataVar): 
    cont=0    
    if nameVar in 'vacio' or dataVar in 'vacio':
        print("vacio")
    else:
        for dato in general:           
            if(type(dato[nameVar])==int):
                dataVar=int(dataVar)
            if(dato[nameVar]==dataVar):
                break       
            cont+=1
    return cont
        

In [26]:
#----------PASO 11. Se encola el mensaje de respuesta en RabbitMQ----------
def envia(ch,props, method, data_string):  
    corr_id=str(props.correlation_id)
    try:
        ch.basic_publish(exchange='',
                     routing_key=props.reply_to,
                     properties=pika.BasicProperties(correlation_id =corr_id
                                                    ),
                     body=str(data_string))
        print("mensaje enviado \n",str(props.reply_to))
    except Exception as error:
        print("mensaje no enviado...",str(error))
        
    ch.basic_ack(delivery_tag = method.delivery_tag)

In [27]:
#-----PASO 9. Función que verifica el tipo de bloque al que pertenece el siguiente estado-----
def confirma(ch,props, method, state, mensaje):
    cont=1
    data_string=' '
    print("Siguiente estado...", state)
    #-------PASO 9. Valdaciones del tipo de bloque al que pertenece el siguiente estado-------
    if((str(consulta("select blocktype from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'"))) not in "None"):
        if(state=="Salida"):
            print("Terminando conversación...")
            cont=-100
        #-------PASO 9. Llamada a la funcion que genera la respuesta del tipo bloque informativo-------
        resp=mensInf(state,mensaje)
        #-------PASO 10. Bloque informativo como respuesta al mensaje del usuario-------
        data_string = json.dumps(resp)        

    elif((str(consulta("select blocktype from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'"))) not in "None"):
        #-------PASO 9. Llamada a la funcion que genera la respuesta del tipo bloque QuickReply-------
        resp=mensQr(state,mensaje)
        #-------PASO 10. Bloque QuickReply como respuesta al mensaje del usuario-------
        data_string = json.dumps(resp)
    
    elif((str(consulta("select blocktype from bloqueinput where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'"))) not in "None"):
        #-------PASO 9. Llamada a la funcion que genera la respuesta del tipo bloque Input-------
        resp=mensInp(state,mensaje)
        #-------PASO 10. Bloque Input como respuesta al mensaje del usuario-------
        data_string = json.dumps(resp)
        
    elif((str(consulta("select blocktype from bloqueinformativoDinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'"))) not in "None"):
        #-------PASO 9. Llamada a la funcion que genera la respuesta del tipo bloque Informativo Dinámico-------
        resp=mensInfDin(state,mensaje)
        #-------PASO 10. Bloque informativo dinámico como respuesta al mensaje del usuario-------
        data_string = json.dumps(resp)
        
    elif((str(consulta("select blocktype from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'"))) not in "None"):
        #-------PASO 9. Llamada a la funcion que genera la respuesta del tipo bloque Input Dinámico-------
        resp=mensInpDin(state,mensaje)
        #-------PASO 10. Bloque input dinámico como respuesta al mensaje del usuario-------
        data_string = json.dumps(resp)
        
    elif((str(consulta("select blocktype from bloqueqrdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'"))) not in "None"):
        #-------PASO 9. Llamada a la funcion que genera la respuesta del tipo bloque QuickReply Dinámico-------
        resp=mensQRDin(state,mensaje)
        #-------PASO 10. Bloque QuickReply dinámico como respuesta al mensaje del usuario-------
        data_string = json.dumps(resp)
    
    elif((str(consulta("select blocktype from bloqueslide where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'"))) not in "None"):
        #-------PASO 9. Llamada a la funcion que genera la respuesta del tipo bloque Slide-------
        resp=mensSlide(state,mensaje)
        #-------PASO 10. Bloque Carrusel como respuesta al mensaje del usuario-------
        data_string = json.dumps(resp)
    
    elif((str(consulta("select blocktype from bloqueslidebuy where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'"))) not in "None"):
        #-------PASO 9. Llamada a la funcion que genera la respuesta del tipo bloque Slide Buy-------
        resp=mensSlideBuy(state,mensaje)
        #-------PASO 10. Bloque Carrusel de compras dinámico como respuesta al mensaje del usuario-------
        data_string = json.dumps(resp)
    
    elif((str(consulta("select blocktype from bloqueticket where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'"))) not in "None"):
        #-------PASO 9. Llamada a la funcion que genera la respuesta del tipo bloque Ticket-------
        resp=mensTicket(state,mensaje)
        #-------PASO 10. Bloque Ticket como respuesta al mensaje del usuario-------
        data_string = json.dumps(resp)
        
    elif((str(consulta("select blocktype from bloqueslidedinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+state+"'"))) not in "None"):
        #-------PASO 9. Llamada a la funcion que genera la respuesta del tipo bloque Slide Dinámico-------
        resp=mensSlideDin(state,mensaje)
        #-------PASO 10. Bloque Carrusel dinámico como respuesta al mensaje del usuario-------
        data_string = json.dumps(resp)
    
    print('\n_________________________________________________________________________________________________________________________')
    print(" [x] Enviando... ", str(data_string))
    #-------PASO 11. Inicio: Llamada a la función encargada de encolar el mensaje de respuesta-------
    envia(ch,props, method, data_string)
    #print("llegó esto: ",mensaje["payload"])
    return cont #,mensaje;

In [28]:
#---------------------PASO 7. Función que realiza las llamadas a APIs de NLP------------------------
def NLP(id_robot,cadena):
    URL=consulta("select api_nlp from robots where id_robot='"+str(id_robot)+"'")
    #---------PASO 7. Validacion que determina si el chatbot cuenta con una API de NLP o no---------
    if URL[0] not in "Sin Api":
        mensaje={"mensaje": cadena}
        URL_API_NLP=URL[0]+str(mensaje)
        print('Operacion get-> ',URL_API_NLP)
        #---------------------PASO 7. Operacion de tipo GET sobre la API de NLP---------------------
        respuesta = requests.get(URL_API_NLP)
        estado = json.loads(respuesta.text)
        print('Salto en la maquina de estados->',estado['salto_estado'])
        #---------------------PASO 7. Retorno de la respuesta de la API de NLP----------------------
        return estado['salto_estado']
    #---------------PASO 7. Retorno que indica que el chatbot no cuenta con API de NLP--------------
    return "Sin estado"

In [29]:
#-----PASO 4. Función que valida que el mensaje introducido por el usuario sea o no un comando------
def comandos(mensaje):
    content='false'
    #Comando para eliminar el registro de un usuario--------------
    if mensaje['payload']['content']=='#DLTU':
        print('Comando #DLTU')
        insertar("delete from ticket where id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
        content = 'Se eliminaron los datos del usuario con id='+mensaje["payload"]["recipient_id"]
    #Comando para obtener la informacion del registro de un usuario------
    elif mensaje['payload']['content']=='#QRYU':
        print('Comando #QRYU')
        data_usuario=consulta2("select * from ticket where id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
        if len(data_usuario)>0:
            content='Datos usuario-> id_robot '+str(data_usuario[0][1])+', id_usuario='+str(data_usuario[0][2])+', estado_actual='+str(data_usuario[0][3])+', entities='+str(data_usuario[0][5])+', ult_actividad= '+str(data_usuario[0][6])
        else:
            content='El usuario no tiene datos almacenados'
    #--------------PASO 4. Retorna la cadena que indica el resultado de la validacion----------------
    return content

In [30]:
#-------------Conexión con RabbitMQ----------------------------
queue_name = 'Colaprueba22'
connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
channel = connection.channel()
channel.queue_declare(queue=queue_name) 
#-------------------PASO 1. Función que se ejecuta cuando se desencola un mensaje.----------------------------
def on_request(ch, method, props, body):
    mensaje = json.loads(body)
    print('\n_________________________________________________________________________________________________________________________')
    print("Mensaje User => ",mensaje)
    print("idFace",mensaje["payload"]["id_face"])
    #---------------PASO 2. Consulta para obtener los datos "robot" y "access_token"--------------------------
    mensaje["payload"]["robot"],mensaje["payload"]["access_token"]=consulta("select id_robot,access_token from robots where id_face='"+str(mensaje["payload"]["id_face"])+"'")
    print("Robot-> ",mensaje["payload"]["robot"],", recipient_id-> "+mensaje["payload"]["recipient_id"])
    #----------------------PASO 3. Consulta para obtener el dato "block_type"---------------------------------
    block_type=(consulta("select block_type from ticket where id_robot="+str(mensaje["payload"]["robot"])+" and id_usuario='"+mensaje["payload"]["recipient_id"]+"'"))    
    #-----PASO 4. Inicio: Se llama a la función comandos, para la validación de los comandos del sistema------
    resp_comand = comandos(mensaje)
    #-----------------------PASO 4. Validación de la respuesta de la función "comandos"-----------------------
    if  resp_comand not in 'false':
        #------PASO 4. Generación y envío de la respuesta para indicar que se llevó acabo la ejecución del comando-------
        resp=mensInfDatosU(mensaje,resp_comand)
        data_string = json.dumps(resp)
        print('\n_________________________________________________________________________________________________________________________')
        print(" [x] Enviando... ", str(data_string))
        #---------------------------PASO 11. Inicio: envío del mensaje de respuesta generado-----------------------------
        envia(ch,props, method, data_string)
    #-------------------------------PASO 5. Inicio: Validación de estado inicial------------------------------
    elif(str(block_type) in "None"):
        #--------------------PASO 5. Generacion de la respuesta mediante el bloque inicial del chatbot-------------------
        n=  {"Respuesta": {"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":[]}}
        data_string = json.dumps(n)
        decoded = json.loads(data_string)
        
        #----------------Creación del JSON que almacena las conversaciones---------
        msj=  {"type_user": "person","id_sender":mensaje["payload"]["recipient_id"], "message": mensaje["payload"]["content"], "variables": "{}", "fecha": str(time.strftime("%d/%m/%y")), "hora": str(time.strftime("%H:%M:%S"))}
        datos_mensaje = json.dumps(msj)
        
        mensajes_= {"conversacion": [] }
        datos_mensajes = json.dumps(mensajes_)
        mensajes = json.loads(datos_mensajes)
        mensajes["conversacion"].append(json.loads(datos_mensaje))
        
        print('id_robot: ',mensaje["payload"]["robot"])
        bloque_inicial,type_blocki=consulta("select block_ini, type_blocki from robots where id_robot='"+str(mensaje["payload"]["robot"])+"'")
        insertar("insert into ticket (id_robot, id_usuario, estado_actual, block_type ,entities,fecha_mod) values ('"+str(mensaje["payload"]["robot"])+"','"+mensaje["payload"]["recipient_id"]+"','"+bloque_inicial+"','"+type_blocki+"','{}','2019/02/06')")
        print(str(mensaje["payload"]["recipient_id"]))
        print(mensajes)
        print("insert into conversaciones (id_ticket, conversacion, fecha_creacion, fecha_termino) values ('"+str(mensaje["payload"]["recipient_id"])+"','"+json.dumps(mensajes)+"','"+time.strftime("%d/%m/%y")+"','2019/02/06')")
        
        insertar("insert into conversaciones (id_ticket, conversacion, fecha_creacion, fecha_termino) values ('"+str(mensaje["payload"]["recipient_id"])+"','"+json.dumps(mensajes)+"','"+time.strftime("%d/%m/%y")+"','2019/02/06')")
        
        decoded["Respuesta"]["content"] = str(consulta("select contenido from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+bloque_inicial+"'")[0])
        decoded["Respuesta"]["next_id"] = str(consulta("select next_id from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+bloque_inicial+"'")[0])
        decoded["Respuesta"]["blockType"]= str(consulta("select blockType from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+bloque_inicial+"'")[0])
        decoded["Respuesta"]["contentType"]= str(consulta("select contentType from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+bloque_inicial+"'")[0])
        decoded["Respuesta"]["typingTime"]= str(consulta("select typingTime from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+bloque_inicial+"'")[0])
        
        payload={"content":"","state":bloque_inicial,"recipient_id":mensaje["payload"]["recipient_id"],"entities":mensaje["payload"]["entities"],"access_token":mensaje["payload"]["access_token"]}
        pay=json.dumps(payload)
        decoded["Respuesta"]["payload"].append(json.loads(pay))
        data_string = json.dumps(decoded)

        print(" [x] Enviando... ", str(data_string))
        #---------------------------PASO 11. Inicio: envío del mensaje de respuesta generado-----------------------------
        envia(ch, props, method,data_string)
    else:
        #----------------------PASO 6. Consulta para obtener los datos "estado_actual" y "entities"----------------------
        estado_actual,entitiesConsulta=consulta("select estado_actual,entities from ticket where id_robot='"+str(mensaje["payload"]["robot"])+"' and id_usuario='"+mensaje["payload"]["recipient_id"]+"'")
        entitiesDB = json.loads(entitiesConsulta)
        mensaje["payload"]["entities"]=entitiesDB
        
        #----------------Actualizacion del JSON que almacena las conversaciones---------
        msj=  {"type_user": "person","id_sender":mensaje["payload"]["recipient_id"], "message": mensaje["payload"]["content"], "variables": entitiesDB, "fecha": str(time.strftime("%d/%m/%y")), "hora": str(time.strftime("%H:%M:%S"))}
        datos_mensaje = json.dumps(msj)
        
        #id_conversacion,convers=consulta("select id_conversacion,conversacion from conversaciones where id_conversacion=(select max(id_conversacion) from conversaciones where id_ticket='"+str(mensaje["payload"]["recipient_id"])+"')")
        #datos_mensajes = json.dumps(convers)
        #mensajes = json.loads(datos_mensajes)
        #mensajes["conversacion"].append(json.loads(datos_mensaje))
        
        #----------------PASO 7. Inicio: Validación del tipo de bloque al que pertenece el estado actual-----------------
        #----------------PASO 7. Acciones y validaciones que se realizan para los bloques "informativo"------------------
        if(block_type[0]=="informativo"):
            #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 1.-------------------
            print("select next_id from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")
            state=str(consulta("select next_id from bloqueinformativo where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0])
            #-----PASO 9. Inicio: Llamada a la función que verifica el tipo de bloque al que pertenece el siguiente estado-----
            cont=confirma(ch, props, method, state, mensaje)
        
        #-----------------PASO 7. Acciones y validaciones que se realizan para los bloques "quickReply"------------------
        if(block_type[0]=="quickReply"):
            opciones=consulta("select opciones from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0].split(",")
            cont=0
            for opcion in opciones:
                if opcion in mensaje["payload"]["content"]:
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 1.-------------------
                    next_id=consulta("select next_id from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0].split(",")
                    if len(next_id)>1:
                        state=str(next_id[cont])
                    else:
                        #save_var=consulta("select save_var from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0]
                        save_var= consulta("select var from bloquequickreply join variables on variables.id_var =  cast ( bloquequickreply.id_var as INTEGER) where bloquequickreply.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueinput.namestate='"+state+"'")[0]
                        mensaje["payload"]["entities"][save_var]=mensaje["payload"]["content"]
                        state=str(next_id[0])
                    break
                cont=cont+1
            else:
                #-------------PASO 7. Llamada a la función que realiza las llamadas a APIS de NLP---------------
                resp=NLP(mensaje['payload']['robot'],mensaje['payload']['content'])
                if resp not in "Sin estado":
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 2.-------------------
                    state=resp
                else:
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 3.-------------------
                    state=str(consulta("select default_id from bloquequickreply where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0])
            #-----PASO 9. Inicio: Llamada a la función que verifica el tipo de bloque al que pertenece el siguiente estado-----
            cont=confirma(ch,props, method, state,mensaje)
        
        #-------------------PASO 7. Acciones y validaciones que se realizan para los bloques "input"---------------------
        elif(block_type[0]=="input"):
            dataVar=mensaje["payload"]["content"]
            #nameVar=str(consulta("select save_var from bloqueinput where id_robot='"+str(+mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0])
                        #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 1.-------------------
            nameVar = str(consulta("select var from bloqueinput join variables on variables.id_var =  cast ( bloqueinput.id_var as INTEGER) where bloqueinput.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueinput.namestate='"+estado_actual+"'")[0])
            mensaje["payload"]["entities"][nameVar]=dataVar
            state=str(consulta("select next_id from bloqueinput where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0])

            #-----PASO 9. Inicio: Llamada a la función que verifica el tipo de bloque al que pertenece el siguiente estado-----
            cont=confirma(ch,props, method, state,mensaje)
            
        
        #------------PASO 7. Acciones y validaciones que se realizan para los bloques "informativoDinamico"--------------
        elif(block_type[0]=="informativoDinamico"):
            #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 1.-------------------
            state=str(consulta("select next_id from bloqueinformativoDinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0])
            #-----PASO 9. Inicio: Llamada a la función que verifica el tipo de bloque al que pertenece el siguiente estado-----
            cont=confirma(ch,props, method, state,mensaje)
            
        
        #---------------PASO 7. Acciones y validaciones que se realizan para los bloques "inputDinamico"-----------------
        elif(block_type[0]=="inputDinamico"):
            dataVar=mensaje["payload"]["content"]
            
            #save_var=consulta("select save_var from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0]
            save_var= str(consulta("select var from bloqueinputdinamico join variables on variables.id_var =  cast ( bloqueinputdinamico.id_var as INTEGER) where bloqueinputdinamico.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueinputdinamico.namestate='"+state+"'")[0])
            if(mensaje["payload"]["entities"][save_var]==" "):
                nameVar=save_var
            else:
                nameVar=mensaje["payload"]["entities"][save_var]
            mensaje["payload"]["entities"][nameVar]=dataVar
            #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 1.-------------------
            state=str(consulta("select next_id from bloqueinputdinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0])
            #-----PASO 9. Inicio: Llamada a la función que verifica el tipo de bloque al que pertenece el siguiente estado-----
            cont=confirma(ch,props, method, state,mensaje)
            
        #-------------PASO 7. Acciones y validaciones que se realizan para los bloques "quickReplyDinamico"--------------
        elif(block_type[0]=="quickReplyDinamico"):
            opciones=consulta("select opciones from bloquequickreplydinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0].split(",")
            cont=0
            for opcion in opciones:
                if opcion in mensaje["payload"]["content"]:
                    dataVar=mensaje["payload"]["content"]
                    #nameVar=consulta("select save_var from bloquequickreplydinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0]
                    nameVar= consulta("select var from bloquequickreplydinamico join variables on variables.id_var =  cast ( bloquequickreplydinamico.id_var as INTEGER) where bloquequickreplydinamico.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloquequickreplydinamico.namestate='"+estado_actual+"'")[0]
                    mensaje["payload"]["entities"][nameVar]=dataVar
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 1.-------------------
                    next_id=consulta("select next_id from bloquequickreplydinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0].split(",")
                    state=str(next_id[cont])
                    break
                cont=cont+1
            else:
                #-------------PASO 7. Llamada a la función que realiza las llamadas a APIS de NLP---------------
                resp=NLP(mensaje['payload']['robot'],mensaje['payload']['content'])
                if resp not in "Sin estado":
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 2.-------------------
                    state=resp
                else:
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 3.-------------------
                    state=str(consulta("select default_id from bloquequickreplydinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0])
            #-----PASO 9. Inicio: Llamada a la función que verifica el tipo de bloque al que pertenece el siguiente estado-----
            cont=confirma(ch,props, method, state,mensaje)
        
        #-------------------PASO 7. Acciones y validaciones que se realizan para los bloques "slide"---------------------
        elif(block_type[0]=="slide"):
            opciones,default_id=consulta("select next_id, default_id from bloqueslide where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")
            s_v = consulta("select var from bloqueslide join variables on variables.id_var =  cast ( bloqueslide.id_var as INTEGER) where bloqueslide.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueslide.namestate='"+estado_actual+"'")
            save_var=s_v.split(",")
            if mensaje["payload"]["type_resp"]=="postback": 
                for opcion in opciones.split(","):
                    if opcion in mensaje["payload"]["content"]:
                        #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 1.-------------------
                        state=opcion
                        break
                else:
                    if len(save_var)>1:
                        #En este caso se almacena en el campo save_var de la DB el nombre de la variable y el next_id, esto cuando en el slide se retorne un dato que se tiene que almacenar y no represente un next_id
                        mensaje["payload"]["entities"][save_var[0]]=mensaje["payload"]["content"]
                        state=save_var[1]
                    else:
                        mensaje["payload"]["entities"][save_var[0]]=mensaje["payload"]["content"]
                        state=opciones
            else:
                #-------------PASO 7. Llamada a la función que realiza las llamadas a APIS de NLP---------------
                resp=NLP(mensaje['payload']['robot'],mensaje['payload']['content'])
                if resp not in "Sin estado":
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 2.-------------------
                    state=resp
                else:
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 3.-------------------
                    state=default_id
            #-----PASO 9. Inicio: Llamada a la función que verifica el tipo de bloque al que pertenece el siguiente estado-----
            cont=confirma(ch,props, method, state,mensaje)
        
        #------------------PASO 7. Acciones y validaciones que se realizan para los bloques "slideBuy"-------------------
        elif(block_type[0]=="slideBuy"):
            elements,next_id,default_id=consulta("select id_elements,next_id,default_id from bloqueslidebuy where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")
            nameVar = consulta("select var from bloqueslidebuy join variables on variables.id_var =  cast ( bloqueslidebuy.id_var as INTEGER) where bloqueslidebuy.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueslidebuy.namestate='"+estado_actual+"'")[0]
            #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 1.-------------------
            state=next_id
            for element in elements.split(","):
                if element in mensaje["payload"]["content"]:                
                    if nameVar in mensaje["payload"]["entities"]:
                        if mensaje["payload"]["content"] in mensaje["payload"]["entities"][nameVar]:
                            #---------------Agrega y suma el nuevo elemeno a comprar seleccionado------------
                            val=mensaje["payload"]["entities"][nameVar][mensaje["payload"]["content"]]
                            mensaje["payload"]["entities"][nameVar][mensaje["payload"]["content"]]=val+1
                            break
                        mensaje["payload"]["entities"][nameVar][mensaje["payload"]["content"]]=1
                        break
                    mensaje["payload"]["entities"][nameVar]={mensaje["payload"]["content"]:1}#--Primer elemeno seleccionado de ese tipo
                    break
            else:
                #-------------PASO 7. Llamada a la función que realiza las llamadas a APIS de NLP---------------
                resp=NLP(mensaje['payload']['robot'],mensaje['payload']['content'])
                if resp not in "Sin estado":
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 2.-------------------
                    state=resp
                else:
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 3.-------------------
                    state=default_id
            #-----PASO 9. Inicio: Función que verifica el tipo de bloque al que pertenece el siguiente estado-----
            cont=confirma(ch,props, method, state,mensaje)
        
        #---------------PASO 7. Acciones y validaciones que se realizan para los bloques "slideDinamico"-----------------
        elif(block_type[0]=="slideDinamico"):
            opciones,default_id=consulta("select next_id, default_id from bloqueslidedinamico where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")
            s_v = consulta("select var from bloqueslidedinamico join variables on variables.id_var =  cast ( bloqueslidedinamico.id_var as INTEGER) where bloqueslidedinamico.id_robot='"+str(mensaje["payload"]["robot"])+"' and bloqueslidedinamico.namestate='"+estado_actual+"'")[0]
            save_var=s_v.split(",")
            if mensaje["payload"]["type_resp"]=="postback":  
                for opcion in opciones.split(","):
                    if opcion in mensaje["payload"]["content"]:
                        #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 1.-------------------
                        state=opcion
                        break
                else:
                    if len(save_var)>2:
                        #En este caso se almacena en el campo save_var de la DB el nombre de la variable y el next_id, esto cuando en el slide se retorne un dato que se tiene que almacenar y no represente un next_id
                        mensaje["payload"]["entities"]["VarTipoDato"]=save_var[0]
                        mensaje["payload"]["entities"][save_var[1]]=mensaje["payload"]["content"]
                        state=save_var[2]
                    else:
                        mensaje["payload"]["entities"]["VarTipoDato"]=save_var[0]
                        mensaje["payload"]["entities"][save_var[1]]=mensaje["payload"]["content"]
                        state=opciones
            else:
                #-------------PASO 7. Llamada a la función que realiza las llamadas a APIS de NLP---------------
                resp=NLP(mensaje['payload']['robot'],mensaje['payload']['content'])
                if resp not in "Sin estado":
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 2.-------------------
                    state=resp
                else:
                    #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 3.-------------------
                    state=default_id
            #-----PASO 9. Inicio: Función que verifica el tipo de bloque al que pertenece el siguiente estado-----
            cont=confirma(ch,props, method, state,mensaje)
            
        #------------------PASO 7. Acciones y validaciones que se realizan para los bloques "ticket"---------------------
        elif(block_type[0]=="ticket"):
            #-----------------PASO 8. Generacion del siguiente estado a ejecutar: CASO 1.-------------------
            state=str(consulta("select next_id from bloqueticket where id_robot='"+str(mensaje["payload"]["robot"])+"' and namestate='"+estado_actual+"'")[0])
            #-----PASO 9. Inicio: Función que verifica el tipo de bloque al que pertenece el siguiente estado-----
            cont=confirma(ch,props, method, state,mensaje)

#-----Creacion de la cola de mensajes en RabbitMQ-----
channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue_name,on_request)
#-----PASO 1.- Inicio: Desencolar mensaje: se llama a la funcion on_request pasando el mensaje descolado-----
print(" [x] Comenzando conversación...")
channel.start_consuming()

 [x] Comenzando conversación...

_________________________________________________________________________________________________________________________
Mensaje User =>  {'payload': {'robot': 'none', 'id_face': '2153123008120093', 'recipient_id': '2374218029278335', 'access_token': 'none', 'content': 'Hola', 'type_resp': 'message', 'state': 'none', 'blocktype': 'none', 'opciones': 'none', 'entitites': ' ', 'entities': {}}}
idFace 2153123008120093
Robot->  121 , recipient_id-> 2374218029278335
select next_id from bloqueinformativo where id_robot='121' and namestate='comprar2'
Siguiente estado... Saludo2

_________________________________________________________________________________________________________________________
 [x] Enviando...  {"Respuesta": {"content": "Hola, espero que estes teniendo un excelente dia.", "next_id": "info_bot2", "blockType": "informativo", "contentType": "tex", "typingTime": "1", "payload": {"content": "", "state": "Saludo2", "recipient_id": "23742180292


_________________________________________________________________________________________________________________________
Mensaje User =>  {'payload': {'robot': 'none', 'id_face': '2153123008120093', 'recipient_id': '2374218029278335', 'access_token': 'none', 'content': 'Queja 2', 'type_resp': 'postback', 'state': 'none', 'blocktype': 'none', 'opciones': 'none', 'entitites': ' ', 'entities': {}}}
idFace 2153123008120093
Robot->  121 , recipient_id-> 2374218029278335
Siguiente estado... Queja2

_________________________________________________________________________________________________________________________
 [x] Enviando...  {"Respuesta": {"content": "Introduce tu queja", "next_id": "Saludo2", "blockType": "input", "contentType": "text", "typingTime": "3", "payload": {"content": "", "state": "Queja2", "recipient_id": "2374218029278335", "entities": {}, "access_token": "EAAHNkscg2TcBAPwb5WSaxOM7IjDf57ZAoYu5TwDdUckaTK7dJc6nWq3fCaZBcKZBEZB3jHZCvbykRpYZAzpUHfIwmZCa45OcnXzND1FZBJNhaW

KeyboardInterrupt: 